In [1]:
# 导入工具包
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import SimpleRNN, LSTM, GRU
from keras.optimizers import SGD, Nadam, Adam, RMSprop
from keras.callbacks import TensorBoard
from keras.utils import np_utils
import scipy.io 
import numpy as np

Using TensorFlow backend.


# 第一步：导入数据

In [2]:
data = scipy.io.loadmat('DATA\sp1s_aa_1000Hz.mat')
y_test = np.loadtxt('DATA\labels of the test set.txt',encoding="utf-8")

# 第二步：预处理数据

In [3]:
"""
将训练数据调整为LSTM的正确输入尺寸
并将数据转换为float 32
"""
x_train = data['x_train'].reshape((316,500,28))
x_train /= 200
x_train = x_train.astype('float32')
"""
将测试数据调整为LSTM的正确输入尺寸
并将数据转换为float 32
"""
x_test = data['x_test'].reshape((100,500,28))
x_test /= 200
x_test = x_test.astype('float32')

"""
将标签数据调整为LSTM的正确输入尺寸
并将数据转换为float 32
"""
y_train = data['y_train'].reshape(316,1)
tmp_train = []
for i in y_train:
    if i == 1:
        tmp_train.append(1)
    elif i == 0:
        tmp_train.append(-1)
y_train = np.array(tmp_train)
y_train = np_utils.to_categorical(y_train, 2)
y_train = y_train.astype('float32')

y_test = y_test.reshape(100,1)
tmp_test = []
for i in y_test:
    if i == 1:
        tmp_test.append(1)
    elif i == 0:
        tmp_test.append(-1)       
y_test = np.array(tmp_test)
y_test = np_utils.to_categorical(y_test, 2)
y_test = y_test.astype('float32')

In [4]:
# 第三步：构建训练模型

In [5]:
model = Sequential()
model.add(LSTM(10, return_sequences = True, input_shape=(500, 28)))
model.add(LSTM(10, return_sequences = True))
model.add(LSTM(5))
model.add(Dense(2, activation = 'softmax'))

model.summary()

"""
优化器设置
学习率为0.001
"""
optim = Nadam(lr = 0.001)
# 设置损失函数为交叉熵损失函数
model.compile(loss = 'categorical_crossentropy', optimizer = optim, metrics = ['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 500, 10)           1560      
_________________________________________________________________
lstm_2 (LSTM)                (None, 500, 10)           840       
_________________________________________________________________
lstm_3 (LSTM)                (None, 5)                 320       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 12        
Total params: 2,732
Trainable params: 2,732
Non-trainable params: 0
_________________________________________________________________


In [6]:
# 第四步：训练模型

In [7]:
"""

epochs设置为10
batch_size设置为20

"""

model.fit(x_train, y_train, epochs=15, batch_size=20)


Epoch 1/15
316/316 [==============================] - 7s 23ms/step - loss: 0.6223 - accuracy: 0.8544
Epoch 2/15
316/316 [==============================] - 6s 20ms/step - loss: 0.3531 - accuracy: 1.0000
Epoch 3/15
316/316 [==============================] - 6s 18ms/step - loss: 0.1842 - accuracy: 1.0000
Epoch 4/15
316/316 [==============================] - 6s 18ms/step - loss: 0.1262 - accuracy: 1.0000
Epoch 5/15
316/316 [==============================] - 6s 19ms/step - loss: 0.0928 - accuracy: 1.0000
Epoch 6/15
316/316 [==============================] - 6s 19ms/step - loss: 0.0692 - accuracy: 1.0000
Epoch 7/15
316/316 [==============================] - 6s 19ms/step - loss: 0.0538 - accuracy: 1.0000
Epoch 8/15
316/316 [==============================] - 6s 19ms/step - loss: 0.0438 - accuracy: 1.0000
Epoch 9/15
316/316 [==============================] - 6s 19ms/step - loss: 0.0368 - accuracy: 1.0000
Epoch 10/15
316/316 [==============================] - 6s 20ms/step - loss: 0.0317 - accur

# 第五步：计算最后得分和精度

In [8]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=1)
print('测试得分:', score)
print('测试精度:', acc)

100/100 [==============================] - 9s 87ms/step
测试得分: 0.017455000057816507
测试精度: 1.0
